#  3D Tracking Data Quality Control (QC) Report

This document explains the purpose, methodology, output, and utility of the provided Python script for 3D trajectory evaluation.

---

## a. What is this code about?

This script performs **Core Quality Control (QC)** for **raw 3D joint tracking data** (like those produced by tools such as SLEAP). It acts as a "health report" to assess the quality of the 3D coordinates *before* any post-processing (smoothing, fixing gaps, etc.) is applied.

The primary goal is to measure and report on three crucial metrics for every tracked joint:

1.  **Coverage (%):** How complete the tracking is.
2.  **Jitter (mm/frame):** How jumpy or noisy the tracked position is.
3.  **Longest Gap (ms):** The worst continuous loss of tracking data.

---

## b. How it Works (Methodology)

The process is divided into clear, standardized steps:

### 1. Data Loading and Setup
* **Input Handling:** The `load_3d_any` function automatically detects and loads data from either an **H5 file** (expecting a `tracks` dataset) or an **NPZ file** (handling various structures).
* **Time Reference:** The script uses the `fps_override_hz` value (**120.0 Hz** in the provided example) to convert frame counts into time units (milliseconds), which is essential for reporting gap lengths.

### 2. Unit Conversion (Standardization)
* **Heuristic Check:** It examines the magnitude of the coordinates. If the 99th percentile of the absolute values is less than 2.0, it **assumes the units are meters** and multiplies all coordinates by **1000** to convert them to **millimeters (mm)**.
* **Output Consistency:** All final results for Jitter and Longest Gap are reported in `mm` and `ms`, regardless of the input's original units.

### 3. Metric Calculation (Per-Joint)

| Metric | Calculation Method | Interpretation |
| :--- | :--- | :--- |
| **Coverage (%)** | Counts frames where all (X, Y, Z) coordinates are **finite** (not $\text{NaN}$ or missing) and divides by the total number of frames ($T$). | Higher is better (closer to 100%). |
| **Jitter (mm/frame)** | Calculates the **Euclidean distance** ($\sqrt{\Delta X^2 + \Delta Y^2 + \Delta Z^2}$) between a joint's position in frame $t$ and frame $t+1$. It reports the **median** (typical jump) and **95th percentile** ($\text{p}95$, worst-case jump) of these distances. | Lower is better (closer to $0 \text{ mm}$). |
| **Longest Gap (ms)** | Iterates through the frames to find the **longest continuous sequence of non-valid (missing) data points** (a dropout). This frame count is converted to milliseconds using the FPS ($\text{Time} = \frac{\text{Frames} \times 1000}{\text{FPS}}$). | Lower is better (closer to $0 \text{ ms}$). |

### 4. Output Generation
* Calculated data is saved into three separate **CSV files** (data tables).
* The data is used to generate three descriptive **PNG plots** (bar and scatter plots) for visual inspection.
* A concise **`summary.txt`** file is generated with the median values across all joints.

---

## c. Output it Produces (Output Files)

All files are created inside the specified `out_root` folder:

| Category | File Name | Format |
| :--- | :--- | :--- |
| **Data Tables** | `3d_coverage_by_joint.csv` | CSV |
| | `3d_jitter_by_joint.csv` | CSV |
| | `3d_dropout_by_joint.csv` | CSV |
| **Visualizations** | `plot_coverage_by_joint.png` | PNG |
| | `plot_jitter_by_joint.png` | PNG |
| | `plot_dropout_by_joint.png` | PNG |
| **Session Summary** | `summary.txt` | TXT |

---

## d. What each file means?

| File Name | Meaning | Key Metric |
| :--- | :--- | :--- |
| **`3d_coverage_by_joint.csv`** | Detailed table of how often each joint was successfully tracked (e.g., Joint 1 was tracked 99.8% of the time). | `coverage_pct` |
| **`3d_jitter_by_joint.csv`** | Detailed table of the frame-to-frame movement (noise) for each joint. | `median_delta_mm`, `p95_delta_mm` |
| **`3d_dropout_by_joint.csv`** | Detailed table of the single worst period of missing data for each joint. | `longest_gap_ms` |
| **`plot_coverage_by_joint.png`** | **Bar Chart:** Visual comparison of coverage across all joints (look for high bars). | Coverage (%) |
| **`plot_jitter_by_joint.png`** | **Scatter/Line Plot:** Visual comparison of movement noise (look for low dots/lines). | $\Delta$ per frame (mm) |
| **`plot_dropout_by_joint.png`** | **Bar Chart:** Visual comparison of the worst blackout time (look for low bars). | Longest gap (ms) |
| **`summary.txt`** | A quick, one-line report with the **median** values for all metrics across the *entire session*. | Overall Session Health |

---

## e. Console Output Types and What Each Means?

The console messages provide feedback on the script's progress:

| Output Prefix | Purpose | Interpretation |
| :--- | :--- | :--- |
| `[info]` | Confirms parameters and loaded data dimensions. | **`T`** = Total frames; **`joints`** = Joint names; **`fps`** = Frame rate used. |
| `[units]` | Reports the unit standardization logic. | Confirms whether data was **assumed to be meters** (and multiplied by 1000) or already in **millimeters**. |
| `[OK] wrote:` | Confirms successful file saving. | The data tables and plots have been created in the output folder. |
| `[SUMMARY]` | The final output line of the script. | The **most important quick-check** of overall data quality (e.g., **Median jitter: 0.85 mm/frame**). |
| `[DONE]` | Confirmation of script completion. | Lists the paths to the generated plots. |

---

## f. How this code helps us

This code is essential for **triage and verification** of motion capture data:

* **Identifies Flaws Early:** Since the analysis is on **raw** data (no smoothing), it gives an honest assessment of the fundamental quality of the 3D reconstruction, identifying problems caused by poor camera calibration or occlusion.
* **Pinpoints Worst Joints:** The per-joint reporting allows researchers to instantly see *which specific joints* are the least reliable (e.g., "The left ear coverage is only 60%") and decide whether to ignore that joint or focus manual effort there.
* **Quantifies Noise:** The Jitter metric provides a clear, objective number (in $\text{mm}$) for tracking noise, which is critical for making informed decisions about necessary **data smoothing/filtering** for downstream biomechanical analysis.

## CT Pedestal Computation (Optional Feature)

This notebook appends four pedestal landmarks using the latest CT fiducial coordinates baked directly into the code.

### Marker Mapping

- **Nose tip**: Fiducial `F` corresponds to the tracked nose landmark (LPS mm coordinates: `[15.0423, 86.4624, 114.5245]`).
- **Pedestal corners**: `F_1` … `F_4` are the four pedestal reference points drawn from the same CT session.

### How It Works

1. **Hardcoded Offsets**: Each pedestal uses `offset = pedestal_position - nose_position` computed from the embedded CT coordinates.
2. **Pedestal Trajectories**: On every frame we add the offset to the tracked nose 3D position to form `Pedestal_F_1` … `Pedestal_F_4`.
3. **Integration**: The new joints flow through all metrics (coverage, jitter, dropout) alongside the tracked markers.

### Configuration

- Set `nose_landmark_name` in the code cell to the tracked nose landmark name (e.g., `"F"`).
- Toggle `include_ct_pedestals` to `False` if you need to disable the extra joints.

### Coordinate System Note

The CT fiducials are expressed in LPS (Left-Posterior-Superior) coordinates. No additional transforms are applied before the offsets are combined with the tracked nose landmark; add any required transforms separately if your 3D data uses a different basis.


In [ ]:
# === PHASE 2 • 3D EVALUATION ===============================================
# Core QC for final 3D tracks (no smoothing, no fixing, just truth):
#   1. Coverage (%)            - how often each joint is valid
#   2. Jitter (mm/frame)       - how jumpy each joint is frame-to-frame
#   3. Longest gap (ms)        - worst continuous blackout for each joint
#
# INPUT:
#   data_path:
#       EITHER an NPZ containing 3D joint trajectories
#       OR an H5 containing a 'tracks' dataset shaped (T, K, J, 3)
#
#   fps_override_hz:
#       If not None, we force that FPS (e.g. 120.0).
#       If None, we try to read "FPS" from the file. If still None, we can't
#       convert gap length to ms (we'll still report frames).
#
# OUTPUT FOLDER:
#   out_root/
#       3d_coverage_by_joint.csv
#       3d_jitter_by_joint.csv
#       3d_dropout_by_joint.csv
#
#       plot_coverage_by_joint.png
#       plot_jitter_by_joint.png
#       plot_dropout_by_joint.png
#
#       summary.txt
#
# All numbers are direct from data. No smoothing, no filtering,
# no threshold cutoffs. This is an honest health report of the raw 3D tracks.
# ============================================================================

data_path        = r"C:\Users\Lenovo\Desktop\Sleap Final Predictions\points3d.h5"
out_root         = r"C:\Users\Lenovo\Desktop\Sleap Final Predictions\out_3d_eval"
fps_override_hz  = 120.0   # <-- set 120.0 if you know capture rate; else None

# ========== CT PEDESTAL CONFIGURATION (OPTIONAL) ==========
# The CT fiducial coordinates are embedded below. Set nose_landmark_name to the
# tracked nose landmark name (e.g., "F" if your 3D skeleton keeps the CT label).
# Set include_ct_pedestals=False to skip adding the four pedestal joints.
nose_landmark_name = "F"   # update to your tracked nose joint name, or set to None to skip
include_ct_pedestals = True  # set False to disable pedestal augmentation
# =========================================================

import os, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py

# Hardcoded CT fiducial coordinates in LPS (mm)
CT_NOSE_MARKER_NAME = "F"
CT_NOSE_POSITION_LPS = np.array([
    15.04228687286377,
    86.46243651882577,
    114.52446880550107,
], dtype=float)

CT_PEDESTAL_POSITIONS_LPS = {
    "F_1": np.array([0.0031134106684476137, 47.44228744506836, 150.02127075195313], dtype=float),
    "F_2": np.array([23.697498321533203, 46.341487884521484, 147.47947692871094], dtype=float),
    "F_3": np.array([11.59162425994873, 39.69370450659213, 155.08550427615032], dtype=float),
    "F_4": np.array([12.013157142401923, 36.49899038310201, 178.17802063449454], dtype=float),
}

os.makedirs(out_root, exist_ok=True)


def _load_from_npz(path):
    """
    NPZ loader.
    Returns:
        joints_dict: {name: (T,3)}
        fps: float or None
    """
    z = np.load(path, allow_pickle=True)
    keys = list(z.keys())
    print("[info:npz] keys:", keys)

    joints = {}
    had_nodes_names = False

    # ---- Case A: one big (T,J,3) block ----
    cand = [(k, z[k].shape) for k in keys
            if hasattr(z[k], "ndim")
            and z[k].ndim == 3
            and z[k].shape[-1] == 3]
    if cand:
        # choose array with largest J
        k_big, shp = sorted(cand, key=lambda kv: kv[1][1], reverse=True)[0]
        XYZ = z[k_big]          # (T,J,3)
        T, J, _ = XYZ.shape

        # joint names if provided
        if "nodes" in z and len(z["nodes"]) == J:
            names = [str(s) for s in z["nodes"]]
            had_nodes_names = True
        else:
            names = [f"J{j+1}" for j in range(J)]

        for j, name in enumerate(names):
            joints[name] = XYZ[:, j, :]  # (T,3)

        fps = float(z["FPS"]) if "FPS" in z else None

        # if names are just "J1", "J2"... and we ALSO see semantic keys like
        # 'U3','L3','H3', try to remap (best effort)
        def try_make_human_names(joints_dict, raw_keys, had_nodes):
            if had_nodes:
                return joints_dict
            all_names = list(joints_dict.keys())
            all_generic = all(re.match(r"^J\d+$", n) for n in all_names)
            if not all_generic:
                return joints_dict  # already meaningful
            # guess semantic order from raw_keys (e.g. ["U3","L3","H3"])
            sem = []
            for kk in raw_keys:
                if kk.upper() == "FPS": 
                    continue
                sem.append(kk)
            # dedupe while preserving order
            seen_tmp = set()
            sem_clean = []
            for s in sem:
                if s not in seen_tmp:
                    sem_clean.append(s)
                    seen_tmp.add(s)
            js_sorted = sorted(
                [n for n in all_names if re.match(r"^J\d+$", n)],
                key=lambda x: int(x[1:])
            )
            if len(js_sorted) == len(sem_clean):
                new_joints = {}
                mapping = {js_sorted[i]: sem_clean[i] for i in range(len(js_sorted))}
                for old_name, xyz in joints_dict.items():
                    new_name = mapping.get(old_name, old_name)
                    new_joints[new_name] = xyz
                return new_joints
            return joints_dict

        joints = try_make_human_names(joints, keys, had_nodes_names)
        return joints, fps

    # ---- Case B: separate arrays per joint name ----
    for k in keys:
        arr = z[k]
        if not hasattr(arr, "ndim"):
            continue
        if arr.ndim == 2 and arr.shape[1] == 3:
            joints[k] = arr  # (T,3)
        elif arr.ndim == 3 and arr.shape[-1] == 3:
            # e.g. H3 is (T,3,3) -> split into H31,H32,H33
            T, JJ, _ = arr.shape
            base = re.sub(r"\W+$", "", k)
            for j in range(JJ):
                joints[f"{base}{j+1}"] = arr[:, j, :]
    fps = float(z["FPS"]) if "FPS" in z else None

    if not joints:
        raise RuntimeError("NPZ: couldn't find any (T,3) or (T,J,3) data")
    return joints, fps


def _load_from_h5(path):
    """
    H5 loader for files like points3d.h5 with dataset 'tracks' of shape (T,K,J,3).
    Returns:
        joints_dict: {name: (T,3)}
        fps: None (H5 doesn't have FPS unless you add later)
    """
    with h5py.File(path, "r") as f:
        if "tracks" not in f:
            raise RuntimeError("H5: expected dataset 'tracks' not found.")
        arr = np.array(f["tracks"])  # shape (T,K,J,3)
    if arr.ndim != 4 or arr.shape[-1] != 3:
        raise RuntimeError(f"H5: unexpected tracks shape {arr.shape}, expected (T,K,J,3)")
    T, K, J, _ = arr.shape

    # pick first track (K=1 in your dump)
    xyz = arr[:, 0, :, :]  # (T,J,3)

    joints = {}
    for j in range(J):
        joints[f"J{j+1}"] = xyz[:, j, :]  # (T,3)

    fps = None  # h5 didn't expose FPS
    return joints, fps


# ========== CT MARKER OFFSETS (HARDCODED) ==========
def compute_ct_pedestal_offsets(ct_dir=None, nose_filename=None, pedestal_filenames=None):
    """
    Provide hardcoded CT offsets for the pedestal fiducials.

    Args:
        ct_dir: Unused (kept for backward compatibility).
        nose_filename: Unused.
        pedestal_filenames: Unused.

    Returns:
        Dictionary with:
            - "coordinate_system": LPS coordinate system string
            - "nose_marker": Nose marker name
            - "nose_position": Nose position [x, y, z] in mm
            - "pedestals": {
                marker_name: {
                    "offset": [dx, dy, dz] in mm,
                    "position": [x, y, z] in mm,
                    "filename": synthetic identifier
                }
              }
    """
    pedestals = {}
    for marker_name, pedestal_pos in CT_PEDESTAL_POSITIONS_LPS.items():
        offset = pedestal_pos - CT_NOSE_POSITION_LPS
        pedestals[marker_name] = {
            "offset": offset.tolist(),
            "position": pedestal_pos.tolist(),
            "filename": f"hardcoded::{marker_name}",
        }

    return {
        "coordinate_system": "LPS",
        "nose_marker": CT_NOSE_MARKER_NAME,
        "nose_position": CT_NOSE_POSITION_LPS.tolist(),
        "pedestals": pedestals,
    }


def compute_pedestal_trajectories(nose_trajectory, pedestal_offsets):
    """
    Compute pedestal trajectories from nose landmark trajectory and CT offsets.

    For each frame: pedestal_position = nose_position + pedestal_offset

    Args:
        nose_trajectory: Array of shape (T, 3) where T is number of frames,
            each row is [x, y, z] position of nose landmark
        pedestal_offsets: Dict mapping marker name -> {"offset": [dx, dy, dz], ...}

    Returns:
        Dict mapping marker name to array of shape (T, 3) with pedestal positions
        for each frame
    """
    if nose_trajectory.ndim != 2 or nose_trajectory.shape[1] != 3:
        raise ValueError(f"Expected nose_trajectory shape (T, 3), got {nose_trajectory.shape}")

    trajectories = {}
    for marker_name, info in pedestal_offsets.items():
        offset = np.array(info["offset"])
        if offset.shape != (3,):
            raise ValueError(f"Expected offset shape (3,) for '{marker_name}', got {offset.shape}")
        trajectories[marker_name] = nose_trajectory + offset

    return trajectories
# ================================================


def load_3d_any(path):
    """
    Unified loader:
      - dispatch to NPZ loader or H5 loader
      - apply fps override if provided
      - optionally compute and append pedestal from CT markers
    """
    ext = os.path.splitext(path)[1].lower()
    if ext == ".npz":
        joints, fps = _load_from_npz(path)
    elif ext == ".h5" or ext == ".hdf5":
        joints, fps = _load_from_h5(path)
    else:
        raise RuntimeError(f"Unsupported file extension: {ext}")

    # manual FPS override (e.g. force 120 Hz instead of whatever/None)
    if fps_override_hz is not None:
        fps = float(fps_override_hz)

    # ========== ADD PEDESTALS IF CONFIGURED ==========
    if include_ct_pedestals and nose_landmark_name:
        try:
            ct_pedestal_data = compute_ct_pedestal_offsets()
            pedestal_offsets = ct_pedestal_data["pedestals"]
            print(
                f"[pedestal] Loaded {len(pedestal_offsets)} CT pedestal offsets "
                f"relative to nose '{ct_pedestal_data['nose_marker']}'"
            )

            # Find nose landmark in loaded joints
            nose_key = nose_landmark_name
            if nose_key not in joints:
                matching_names = [n for n in joints.keys() if n.lower() == nose_key.lower()]
                if matching_names:
                    nose_key = matching_names[0]
                    print(f"[pedestal] Matched nose landmark: {nose_key}")
                else:
                    print(f"[pedestal] WARNING: Nose landmark '{nose_landmark_name}' not found in joints.")
                    print(f"[pedestal] Available joints: {list(joints.keys())}")
                    print(f"[pedestal] Skipping pedestal computation.")
                    return joints, fps

            # Get nose trajectory
            nose_traj = joints[nose_key]  # (T, 3)

            # Compute pedestal trajectories
            pedestal_trajs = compute_pedestal_trajectories(nose_traj, pedestal_offsets)

            # Add pedestals to joints dictionary
            added_joint_names = []
            for marker_name, traj in pedestal_trajs.items():
                joint_name = f"Pedestal_{marker_name}"
                joints[joint_name] = traj
                added_joint_names.append(joint_name)

            joined = ", ".join(added_joint_names)
            print(f"[pedestal] Added pedestal trajectories computed from '{nose_key}': {joined}")
            print("[pedestal] Pedestals will appear in all metrics and outputs")
        except Exception as e:
            print(f"[pedestal] ERROR: Failed to compute pedestals: {e}")
            print(f"[pedestal] Continuing without pedestals...")
    # ================================================

    return joints, fps


# ---------- load data ---------- #
joints, fps = load_3d_any(data_path)

names = list(joints.keys())
T = len(next(iter(joints.values())))
print(f"[info] T={T}, joints={names}, fps={fps or 'unknown'}")

# ---------------- Units: convert everything to mm for reporting ------------ #
# Heuristic:
#   If coordinate magnitudes are <~2 units, assume meters -> convert to mm.
#   Else assume already mm.
all_vals = np.concatenate([joints[n].reshape(-1,3) for n in names], axis=0)
finite_mask = np.isfinite(all_vals).all(axis=1)
rng = np.nanpercentile(np.abs(all_vals[finite_mask]), 99)
assume_meters = rng < 2.0
scale_to_mm = 1000.0 if assume_meters else 1.0
print(f"[units] treating as {'meters' if assume_meters else 'millimeters'} "
      f"→ multiplying by {scale_to_mm} to report in mm")

def to_mm(X):
    return X * scale_to_mm

# ---------------- Metric 1: Coverage --------------------------------------- #
cov_rows = []
for n in names:
    xyz = joints[n]  # (T,3)
    valid = np.isfinite(xyz).all(axis=1)  # True if that frame has all coords finite
    cov_pct = 100.0 * valid.mean()
    cov_rows.append(dict(
        joint=n,
        coverage_pct=cov_pct,
        n_valid=int(valid.sum()),
        n_total=T,
    ))
df_cov = pd.DataFrame(cov_rows).sort_values("joint")

# ---------------- Metric 2: Jitter ----------------------------------------- #
def per_frame_delta_mm(xyz):
    diffs = np.diff(to_mm(xyz), axis=0)   # (T-1,3) in mm
    d = np.linalg.norm(diffs, axis=1)    # Euclidean step distance in mm
    d = d[np.isfinite(d)]
    return d

jit_rows = []
for n in names:
    d = per_frame_delta_mm(joints[n])
    if d.size == 0:
        med = np.nan
        p95 = np.nan
    else:
        med = float(np.median(d))
        p95 = float(np.percentile(d, 95))
    jit_rows.append(dict(
        joint=n,
        median_delta_mm=med,
        p95_delta_mm=p95,
        n_pairs=int(d.size),
    ))
df_jit = pd.DataFrame(jit_rows).sort_values("joint")

# ---------------- Metric 3: Longest dropout gap ---------------------------- #
def longest_gap_info(xyz, fps_val):
    valid = np.isfinite(xyz).all(axis=1)  # shape (T,)
    longest_len_frames = 0
    cur_len = 0
    for v in valid:
        if not v:
            cur_len += 1
        else:
            if cur_len > longest_len_frames:
                longest_len_frames = cur_len
            cur_len = 0
    if cur_len > longest_len_frames:
        longest_len_frames = cur_len

    if fps_val is not None and fps_val > 0:
        ms = 1000.0 * (longest_len_frames / fps_val)
    else:
        ms = np.nan
    return longest_len_frames, ms

drop_rows = []
for n in names:
    longest_frames, longest_ms = longest_gap_info(joints[n], fps)
    cov_pct = float(df_cov[df_cov["joint"] == n]["coverage_pct"].iloc[0])
    drop_rows.append(dict(
        joint=n,
        coverage_pct=cov_pct,
        longest_gap_frames=longest_frames,
        longest_gap_ms=longest_ms,
    ))
df_drop = pd.DataFrame(drop_rows).sort_values("joint")

# ---------------- Save CSVs ------------------------------------------------ #
p_cov  = os.path.join(out_root, "3d_coverage_by_joint.csv")
p_jit  = os.path.join(out_root, "3d_jitter_by_joint.csv")
p_drop = os.path.join(out_root, "3d_dropout_by_joint.csv")

df_cov.to_csv(p_cov,  index=False)
df_jit.to_csv(p_jit,  index=False)
df_drop.to_csv(p_drop, index=False)

print("[OK] wrote:", p_cov)
print("[OK] wrote:", p_jit)
print("[OK] wrote:", p_drop)

# ---------------- Plots ---------------------------------------------------- #
def pretty_names(xs):
    return [re.sub(r"\.npz$","", re.sub(r"[_\s]+"," ", x)).strip() for x in xs]

# A) Coverage
plt.figure(figsize=(8,3.0))
order_cov = df_cov.sort_values("coverage_pct", ascending=False)
plt.bar(pretty_names(order_cov["joint"]), order_cov["coverage_pct"])
plt.ylim(0,100)
plt.ylabel("Coverage (%)")
plt.title("3D Coverage by Joint")
plt.xticks(rotation=30, ha="right")
plt.tight_layout()
plt.savefig(os.path.join(out_root, "plot_coverage_by_joint.png"), dpi=150)
plt.close()

# B) Jitter
plt.figure(figsize=(8,3.0))
order_jit = df_jit.sort_values("median_delta_mm", ascending=True)
xpos = np.arange(len(order_jit))
plt.scatter(xpos, order_jit["median_delta_mm"])
for i, (m, p) in enumerate(zip(order_jit["median_delta_mm"],
                                order_jit["p95_delta_mm"])):
    if np.isfinite(m) and np.isfinite(p):
        plt.plot([i,i], [m, p], linewidth=2)
plt.xticks(xpos, pretty_names(order_jit["joint"]), rotation=30, ha="right")
plt.ylabel("Δ per frame (mm)")
plt.title("3D Jitter by Joint (median • p95)")
plt.tight_layout()
plt.savefig(os.path.join(out_root, "plot_jitter_by_joint.png"), dpi=150)
plt.close()

# C) Longest dropout
plt.figure(figsize=(8,3.0))
order_drop = df_drop.sort_values("longest_gap_ms", ascending=False)
plt.bar(pretty_names(order_drop["joint"]), order_drop["longest_gap_ms"])
plt.ylabel("Longest gap (ms)")
plt.title("Worst Continuous Dropout Per Joint (lower is better)")
plt.xticks(rotation=30, ha="right")
plt.tight_layout()
plt.savefig(os.path.join(out_root, "plot_dropout_by_joint.png"), dpi=150)
plt.close()

# ---------------- One-line session summary --------------------------------- #
median_cov_pct     = float(np.nanmedian(df_cov["coverage_pct"]))
median_jitter_mm   = float(np.nanmedian(df_jit["median_delta_mm"]))
median_worst_gapms = float(np.nanmedian(df_drop["longest_gap_ms"])) if fps else np.nan

summary = (
    f"Coverage median: {median_cov_pct:.1f}% | "
    f"Median jitter: {median_jitter_mm:.2f} mm/frame | "
    f"Median worst-gap: {median_worst_gapms:.1f} ms"
)

with open(os.path.join(out_root, "summary.txt"), "w", encoding="utf-8") as f:
    f.write(summary + "\n")

print("[SUMMARY]", summary)
print("[DONE] Plots:",
      os.path.join(out_root,"plot_coverage_by_joint.png"),
      os.path.join(out_root,"plot_jitter_by_joint.png"),
      os.path.join(out_root,"plot_dropout_by_joint.png"),
      sep="\n  ")


[info] T=13357, joints=['J1', 'J2', 'J3', 'J4', 'J5', 'J6', 'J7', 'J8', 'J9', 'J10', 'J11', 'J12', 'J13', 'J14', 'J15', 'J16', 'J17', 'J18', 'J19', 'J20'], fps=120.0
[units] treating as millimeters → multiplying by 1.0 to report in mm
[OK] wrote: C:\Users\Lenovo\Desktop\Sleap Final Predictions\out_3d_eval\3d_coverage_by_joint.csv
[OK] wrote: C:\Users\Lenovo\Desktop\Sleap Final Predictions\out_3d_eval\3d_jitter_by_joint.csv
[OK] wrote: C:\Users\Lenovo\Desktop\Sleap Final Predictions\out_3d_eval\3d_dropout_by_joint.csv
[SUMMARY] Coverage median: 100.0% | Median jitter: 0.10 mm/frame | Median worst-gap: 0.0 ms
[DONE] Plots:
  C:\Users\Lenovo\Desktop\Sleap Final Predictions\out_3d_eval\plot_coverage_by_joint.png
  C:\Users\Lenovo\Desktop\Sleap Final Predictions\out_3d_eval\plot_jitter_by_joint.png
  C:\Users\Lenovo\Desktop\Sleap Final Predictions\out_3d_eval\plot_dropout_by_joint.png
